In [1]:
from seldon_deploy_sdk import Configuration, ApiClient, PredictApi, SeldonDeploymentsApi, ModelMetadataServiceApi, DriftDetectorApi, BatchJobsApi, BatchJobDefinition
from seldon_deploy_sdk.auth import OIDCAuthenticator
from seldon_deploy_sdk.rest import ApiException
import pprint
import json
import requests
from requests.structures import CaseInsensitiveDict

In [2]:
SD_IP = "34.148.148.91"
config = Configuration()
config.host = f"http://{SD_IP}/seldon-deploy/api/v1alpha1"
config.oidc_client_id = "sd-api"
config.oidc_server = f"http://{SD_IP}/auth/realms/deploy-realm"
config.oidc_client_secret = "sd-api-secret"
config.auth_method = 'client_credentials'
config.verify_ssl = False
auth = OIDCAuthenticator(config)
config.id_token = auth.authenticate()
seldon_api_client = ApiClient(configuration=config, authenticator=auth)

In [3]:
print(config.id_token)

eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJseEFOZHlPNjNmQTFGakNHNk1vZjB0Z0tVNVJtWmdvWWd5N0c0Ym5NY3R3In0.eyJleHAiOjE2NjAwOTc0NjYsImlhdCI6MTY2MDA2MTQ2NiwiYXV0aF90aW1lIjowLCJqdGkiOiI0YTQ1NGY5YS1jOWViLTQ3N2MtOTE4MS0xOTA3NTM0ZjY3ZjciLCJpc3MiOiJodHRwOi8vMzQuMTQ4LjE0OC45MS9hdXRoL3JlYWxtcy9kZXBsb3ktcmVhbG0iLCJhdWQiOlsic2QtYXBpIiwiZGVwbG95LXNlcnZlciJdLCJzdWIiOiIxYzM5OGExYS0wNWU1LTRmMDMtYWFmZS02YTRmYzQ0Y2EwZGEiLCJ0eXAiOiJJRCIsImF6cCI6InNkLWFwaSIsInNlc3Npb25fc3RhdGUiOiI2M2M2NjFkOC1jY2UxLTRkNzQtYTg1Ni01NzRiNGM3MGU1OWMiLCJhY3IiOiIxIiwiZW1haWxfdmVyaWZpZWQiOmZhbHNlLCJjbGllbnRJZCI6InNkLWFwaSIsImNsaWVudEhvc3QiOiIxMC4xNDIuMC41MSIsImdyb3VwcyI6W10sInByZWZlcnJlZF91c2VybmFtZSI6InNlcnZpY2UtYWNjb3VudC1zZC1hcGkiLCJjbGllbnRBZGRyZXNzIjoiMTAuMTQyLjAuNTEifQ.e0doEITaHcmvuSTPfuwZVFQYzKxvoT5kD_AJs6VIg_wc1mH9opnA2VIl5BdWwMKkbXIswpsXzitlNwMKgImeCBZi0CMNA1axM_dLhAAUcUQJhJyonXroq0Gy_8OGEX9ZPEp5R_ZImnS3ZJaythQEIFQY-t7KDPapo9Fmd6QwPxGyqNYgRkgmA8uv2iUq3vg9ZUwfbPilvIoj-UrjpIiDc3eTKp2-NBzsDRNp5YAGgG1mhp9EHuG3b6nA0sxOx

In [5]:
DEPLOYMENT_NAME = "iris-accuracy"
NAMESPACE = "seldon-demos"

### Make Prediction - SDK

In [6]:
# create an instance of the API class
predict_api = PredictApi(seldon_api_client)

prediction = {
	"data": {
		"names": ["Sepal length", "Sepal width", "Petal length", "Petal Width"],
		"ndarray": [
			[6.8, 2.8, 4.8, 1.4]
		]
	}
}

try:
    api_response = predict_api.predict_seldon_deployment(DEPLOYMENT_NAME, NAMESPACE, prediction)
    print(api_response)
except ApiException as e:
    print("Exception when calling PredictApi->predict_seldon_deployment: %s\n" % e)

{'data': {'names': ['t:0', 't:1', 't:2'], 'ndarray': [[0.008074020139119268, 0.7781601484223357, 0.21376583143854502]]}, 'meta': {'requestPath': {'iris-accuracy-container': 'seldonio/sklearnserver:1.13.1'}}}


## Make a prediction using the API

In [28]:
URL = f"http://{SD_IP}/seldon/seldon-demos/iris-accuracy/api/v1.0/predictions"
print(URL)

r = requests.post(URL, json=prediction, verify=False)

request_id = r.headers["seldon-puid"]

http://34.148.148.91/seldon/seldon-demos/iris-accuracy/api/v1.0/predictions
755a2125-d0bd-4df4-a023-4ff9cb16e55f


## Bash Example

In [21]:
%%bash

CLUSTER_IP=$(kubectl -n istio-system get service istio-ingressgateway -o jsonpath='{.status.loadBalancer.ingress[0].ip}')
curl -k -s -H "Content-Type: application/json" http://$CLUSTER_IP/seldon/seldon-demos/iris-accuracy/api/v1.0/predictions -d '{"data":{"names":["Sepal length","Sepal width","Petal length","Petal Width"],"ndarray":[[6.8,2.8,4.8,1.4]]}}'  | jq

{
  "data": {
    "names": [
      "t:0",
      "t:1",
      "t:2"
    ],
    "ndarray": [
      [
        0.008074020139119268,
        0.7781601484223357,
        0.21376583143854502
      ]
    ]
  },
  "meta": {
    "requestPath": {
      "iris-accuracy-container": "seldonio/sklearnserver:1.13.1"
    }
  }
}


In [11]:
CLUSTER_IP=$(kubectl -n istio-system get service istio-ingressgateway -o jsonpath='{.status.loadBalancer.ingress[0].ip}')
curl -k -H "Content-Type: application/json" http://$CLUSTER_IP/seldon/seldon-demos/iris-accuracy/api/v0.1/feedback -d '{"response":{"data":{"ndarray":[1]}},"truth":{"data":{"ndarray":[1]}}}'


zsh:1: command not found: kubectl
curl: (6) Could not resolve host: seldon


# Send Feedback of Request

In [90]:

# import requests module
import requests
from requests.structures import CaseInsensitiveDict

URL = f"http://{SD_IP}/seldon/seldon-demos/iris-accuracy/api/v1.0/feedback"
headers = CaseInsensitiveDict()
headers["seldon-puid"] = request_id

body = {
   "response":{
      "data":{
         "ndarray":[
            1
         ]
      }
   },
   "truth":{
      "data":{
         "ndarray":[
            0
         ]
      }
   }
}


r = requests.post(URL, headers=headers, json=body)
  
# print request object
print(r.text)

{"data":{"tensor":{"shape":[0]}},"meta":{"requestPath":{"iris-accuracy-container":"seldonio/sklearnserver:1.13.1"}}}



In [47]:
%%bash

CLUSTER_IP=$(kubectl -n istio-system get service istio-ingressgateway -o jsonpath='{.status.loadBalancer.ingress[0].ip}')
curl -k -s -H "Content-Type: application/json" http://$CLUSTER_IP/seldon/seldon-demos/iris-accuracy/api/v1.0/feedback \
    -d '{"response":{"data":{"ndarray":[1]}},"truth":{"data":{"ndarray":[1]}}}'

{"data":{"tensor":{"shape":[0]}},"meta":{"requestPath":{"iris-accuracy-container":"seldonio/sklearnserver:1.13.1"}}}
